<img src='https://next.ific.uv.es/next/templates/rt_quasar_j15/images/logo/stylenext/logo.png' style='height: 40px' />


# SiPM S1 simulation
In this widget we simulate the sum of the signals coming from SiPMs coupled to wavelength shifting fibers. The fibers are assumed to uniformly cover the longitudinal side of the barrel. The SiPM dark rate is assumed to [increse by a factor of 1.65 every 5.3 K](https://hub.hamamatsu.com/us/en/technical-notes/mppc-sipms/what-are-the-effects-of-temperature-on-dark-count-rates-in-an-SiPM-MPPC.html).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import ceil, exp, pi
import ipywidgets as widgets
np.random.seed(42)

In [2]:
style = {'description_width': 'initial'}
layout = widgets.Layout(width='500px')
detector_diameter = widgets.FloatSlider(description='Detector diameter [m]',
                                        style=style,
                                        layout=layout,
                                        min=2,
                                        max=3,
                                        value=2.5)

fiber_diameter = widgets.IntSlider(description='Fiber diameter [mm]',
                                   style=style,
                                   layout=layout,
                                   min=1,
                                   max=2)

sipm_type = widgets.Dropdown(options=['1x1', '2x2', '3x3', '6x6', '10x10'],
                             value='3x3',
                             description=r'SiPM area [mm$^2$]',
                             layout=layout,
                             style=style)

n_photons = widgets.IntSlider(description='Number of photons',
                              style=style,
                              layout=layout,
                              min=0,
                              value=1000,
                              max=5000)
efficiency = widgets.FloatSlider(description='Efficiency [%]',
                                 style=style,
                                 layout=layout,
                                 min=0,
                                 value=2,
                                 max=100)
dark_rate = widgets.FloatLogSlider(description=r'SiPM dark rate per mm$^2$ at 20 $^{\circ}$C [MHz]',
                                   layout=layout,
                                   value=0.1,
                                   base=10,
                                   min=-2,
                                   max=1,
                                   step=0.1,
                                   style=style)


temperature = widgets.IntSlider(description=r'Temperature [$^{\circ}$C]',
                                style=style,
                                layout=layout,
                                min=-80,
                                value=20,
                                max=40)

In [3]:
def light_signal(t, t0, f, ts, tt):
    if t-t0 > 0:
        return f * 1./ts * exp(-(t-t0)/ts) + (1-f) * 1./tt * exp(-(t-t0)/tt)
    else:
        return 0

def xenon_light(t, t0):
    f = 0.25
    ts = 0.0043
    tt = 0.0269
    return light_signal(t, t0, f, ts, tt)

v_xenon_light = np.vectorize(xenon_light, otypes=[np.float64])

def gui(sipm_type, n_photons, fiber_diameter, efficiency, detector_diameter, dark_rate, temperature):
    sipm_area = eval(sipm_type.replace('x','*'))
    
    n_pes = n_photons * efficiency / 100
    n_fibers = int(ceil(detector_diameter*pi/(fiber_diameter * 1e-3)))
    n_sipms = int(ceil(n_fibers/(sipm_area/(fiber_diameter**2))))

    sipms = np.zeros((n_sipms,100))
    dark_rate *= sipm_area
    dark_rate_factor = 1.65**((temperature-20)/5.3)
    dark_rate *= dark_rate_factor

    counts = np.random.poisson(dark_rate, n_sipms)
    for ic,c in enumerate(counts):
        sipms[ic][np.random.randint(100, size=c)] = 1

    fig, ax = plt.subplots(1,1,constrained_layout=True)
    x = np.linspace(0,1,100)
    summed_signals = np.sum(sipms,axis=0)
    signal = np.round(n_pes*v_xenon_light(x, 0.5)/x.shape[0])
    summed_signals += signal
    ax.plot(x,signal, label=f'Signal ({n_pes:.2g} p.e.)', c='r', lw=2, ls='--')
    ax.plot(x,summed_signals,label="Signal + dark rate at %i $^{\circ}$C (%.2g [MHz])" % (temperature, dark_rate), lw=2, c='k')

    ax.set_xlabel(r"Time [$\mathrm{\mu}$s]")
    ax.set_ylabel("Counts")
    ax.set_ylim(top=np.max(summed_signals)*1.2)
    ax.legend(loc='upper right')
    fig.suptitle(f'Sum of SiPM S1 signals - {n_fibers} fibers, {n_sipms} SiPMs')

out = widgets.interactive_output(gui, {'sipm_type': sipm_type,
                                       'n_photons': n_photons, 
                                       'fiber_diameter': fiber_diameter,
                                       'efficiency': efficiency, 
                                       'detector_diameter': detector_diameter,
                                       'dark_rate': dark_rate,
                                       'temperature': temperature})
ui = widgets.VBox([sipm_type, dark_rate, temperature, n_photons, fiber_diameter, efficiency, detector_diameter])
display(ui, out)



Output()

<hr>
Stefano Roberto Soleti - <a href="mailto:roberto.soleti@dipc.org">roberto.soleti@dipc.org</a>